

<h1 align=center><font size = 5>Final Project Chicago SW suburbs analysis for best location to open a new gym</font></h1>


download and import all the dependencies that we will need.


<a id='item1'></a>


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

print('Libraries imported.')

Libraries imported.


In [2]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [3]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
! pip install folium==0.5.0

import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 79 kB 6.7 MB/s eta 0:00:011
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=283749d834c0c9689184dc0a8f08da10b445ad051266444c2b8c515d42c7e8af
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Libraries imported.


## 1. Download and Explore Datasets


#### Read Chicago southwest suburb zip code data from the web url.
#### Source: Capitol Impact http://www.ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?zip=604


In [80]:
url='http://www.ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?zip=604'

dfs=pd.read_html(url)
mydf1=dfs[2]
mydf2=dfs[3]
mydf3=dfs[4]

#Data is in 3 tables so combine them into one
mydf12=mydf1.append(mydf2, ignore_index = True)
mydf4=mydf12.append(mydf3, ignore_index = True)

#Drop duplicate zip codes
myZipCodes=mydf4.drop_duplicates(subset="Zip Code").reset_index(drop=True)

myZipCodes['Zip Code']=myZipCodes['Zip Code'].apply(str)
myZipCodes['Zip Code']=myZipCodes['Zip Code'].str.strip()
myZipCodes.head()


,Zip Code,City,County
0,60401,Beecher,Will County
1,60402,Forestview,Cook County
2,60406,Blue Island,Cook County
3,60407,Braceville,Grundy County
4,60408,Braidwood,Will County


#### Read Illinois zip code coordinate data from the web url. 
#### Source: Open Data Soft https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/?refine.state=IL

In [81]:
#url='https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/?refine.state=IL'
url='https://gist.github.com/erichurst/7882666'
dfs=pd.read_html(url)
mydfs1=dfs[0]

# Clean data, everything is in the second column.
# Drop First column and parse second column into seperate fields
mydfs2=mydfs1.drop(0, axis=1)
cn=['zip code', 'latitude', 'longitude']
mydfs3=pd.DataFrame(columns=cn)
mydfs3[['zip code', 'latitude', 'longitude']] = mydfs2[1].str.split(',', expand=True)
mydfs3['zip code']=mydfs3['zip code'].str.strip()
mydfs3.head()

,zip code,latitude,longitude
0,ZIP,LAT,LNG
1,00601,18.180555,-66.749961
2,00602,18.361945,-67.175597
3,00603,18.455183,-67.119887
4,00606,18.158345,-66.932911


#### Filter out all but zip codes that being with 604 to get only
#### Chicago southwest suburb zip codes

In [82]:
mydfs4=mydfs3.loc[2:,:]
#mydfs4.head()
swzipcoords=mydfs4[mydfs4['zip code'].str.startswith('604')].reset_index(drop=True)

swzipcoords['latitude']=swzipcoords['latitude'].astype(np.float64)
swzipcoords['longitude']=swzipcoords['longitude'].astype(np.float64)
swzipcoords.head()


,zip code,latitude,longitude
0,60401,41.343966,-87.616389
1,60402,41.835216,-87.791373
2,60403,41.553739,-88.134185
3,60404,41.514026,-88.223228
4,60406,41.654945,-87.681952


#### Create a new data frame chicagosuburbs combining the zip code suburb data with coordinate data

In [83]:
# define the dataframe columns
column_names = ['Zip Code', 'City', 'County', 'Latitude', 'Longitude']

# instantiate the dataframe
chicagosuburbs = pd.DataFrame(columns=column_names)

for zip_code, city, county in zip(myZipCodes['Zip Code'], myZipCodes['City'], myZipCodes['County']):
    # loop until you get the coordinates
    #print(zip_code)
    
    for zi, la, lo in zip(swzipcoords['zip code'], swzipcoords['latitude'], swzipcoords['longitude']):
        if zi == zip_code:
            chicagosuburbs = chicagosuburbs.append({'Zip Code': zip_code,
                                            'City': city,
                                            'County': county,
                                            'Latitude': la,
                                            'Longitude': lo}, ignore_index=True)
chicagosuburbs.head()

,Zip Code,City,County,Latitude,Longitude
0,60401,Beecher,Will County,41.343966,-87.616389
1,60402,Forestview,Cook County,41.835216,-87.791373
2,60406,Blue Island,Cook County,41.654945,-87.681952
3,60407,Braceville,Grundy County,41.234907,-88.261302
4,60408,Braidwood,Will County,41.263739,-88.215536


#### Use geopy library to get the latitude and longitude values of Chicago.


In [84]:
address = 'Chicago, Illinois'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chicago are 41.8755616, -87.6244212.


#### Create a map of Chicago with SW suburbs superimposed on top.

In [85]:
# create map of Chicago and suburbs using latitude and longitude values
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, county, city in zip(chicagosuburbs['Latitude'], chicagosuburbs['Longitude'], chicagosuburbs['County'], chicagosuburbs['City']):
    #print(lat, lng, county, city)
    
    label = '{}, {}'.format(city, county)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  
    
map_chicago

#### Define Foursquare Credentials and Version

In [86]:
CLIENT_ID = '0SSVM0GFP3X1YIMAW2FZMQNUDKPLBRQM3ACB3XVPU1XDWMGI' # your Foursquare ID
CLIENT_SECRET = 'XIU2OIYW2ZZK0W3BQZWHKHJU1UKP2CCNBXWBIK1Q1DU50O3I' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0SSVM0GFP3X1YIMAW2FZMQNUDKPLBRQM3ACB3XVPU1XDWMGI
CLIENT_SECRET:XIU2OIYW2ZZK0W3BQZWHKHJU1UKP2CCNBXWBIK1Q1DU50O3I


#### create a function to get the top 100 venues for each city

In [87]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Create a function to return all the gyms in each city

In [88]:
def getNearbyGyms(names, latitudes, longitudes, radius=2000):
    
    gym_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&query=gym+fitness&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        #results = requests.get(url).json()["response"]['groups'][0]['items']
        results = requests.get(url).json()["response"]["venues"]
        
        # return only relevant information for each nearby venue
        gym_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng']) for v in results])
            

    nearby_gyms = pd.DataFrame([item for gym_list in gym_list for item in gym_list])
    nearby_gyms.columns = ['City', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude']
                
    
    return(nearby_gyms)

#### call get mearby venues function on each suburb and create a new dataframe called chicago_venues.

In [89]:
chicago_venues = getNearbyVenues(names=chicagosuburbs['City'],
                                   latitudes=chicagosuburbs['Latitude'],
                                   longitudes=chicagosuburbs['Longitude']
                                  )

#### call get nearby gyms function on each suburb and create a new dataframe call chicago_gyms

In [90]:
chicago_gyms = getNearbyGyms(names=chicagosuburbs['City'],
                                   latitudes=chicagosuburbs['Latitude'],
                                   longitudes=chicagosuburbs['Longitude']
                                  )

#### Check the size of each

In [91]:
print(chicago_venues.shape)
chicago_venues.head()

(277, 7)


,City,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Beecher,41.343966,-87.616389,BUDDYS Drive-In,41.342946,-87.621050,Fast Food Restaurant
1,Beecher,41.343966,-87.616389,Cardinal Creek Golf Course,41.343563,-87.619754,Golf Course
2,Beecher,41.343966,-87.616389,Dollar General,41.345212,-87.621849,Discount Store
3,Beecher,41.343966,-87.616389,Law Office Of Howard Perry,41.340757,-87.616881,Lawyer
4,Beecher,41.343966,-87.616389,Pin & Tonic Bowling Alley & Sports Bar,41.344149,-87.620711,Bowling Alley


In [92]:
print(chicago_gyms.shape)
chicago_gyms.head()

(283, 6)


,City,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
0,Beecher,41.343966,-87.616389,Fitness Premier,41.351256,-87.620395
1,Beecher,41.343966,-87.616389,Fitness Premier,41.350956,-87.620262
2,Beecher,41.343966,-87.616389,Fitness Premier,41.350703,-87.621310
3,Forestview,41.835216,-87.791373,XSport Fitness,41.819384,-87.804062
4,Forestview,41.835216,-87.791373,The Gym,41.852899,-87.786339


## First Analyze Each Neighborhood

In [94]:
# one hot encoding
chicago_onehot = pd.get_dummies(chicago_venues[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
chicago_onehot['City'] = chicago_venues['City'] 

# move city column to the first column
fixed_columns = [chicago_onehot.columns[-1]] + list(chicago_onehot.columns[:-1])
chicago_onehot = chicago_onehot[fixed_columns]

chicago_onehot.head()

,City,ATM,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Garden,Bistro,Bowling Alley,Breakfast Spot,Brewery,Bus Station,Business Service,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Diner,Discount Store,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Eye Doctor,Fabric Shop,Fast Food Restaurant,Field,Flower Shop,Food,Fried Chicken Joint,Gas Station,Gay Bar,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hot Dog Joint,Hotel,Ice Cream Shop,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Jewelry Store,Lake,Lawyer,Library,Light Rail Station,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Music Venue,New American Restaurant,Nightclub,Nightlife Spot,Optical Shop,Other Repair Shop,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Post Office,Print Shop,Pub,Rental Car Location,Resort,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Supermarket,Sushi Restaurant,Taco Place,Thai Restaurant,Theater,Thrift / Vintage Store,Toll Plaza,Trail,Train Station,Video Store,Weight Loss Center,Wine Bar,Yoga Studio
0,Beecher,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Beecher,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Beecher,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Beecher,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Beecher,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Check the size

In [95]:
chicago_onehot.shape

(277, 118)

#### Group rows by city and by taking the mean of the frequency of occurrence of each category

In [96]:
chicago_grouped = chicago_onehot.groupby('City').mean().reset_index()
chicago_grouped

,City,ATM,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Garden,Bistro,Bowling Alley,Breakfast Spot,Brewery,Bus Station,Business Service,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Diner,Discount Store,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Eye Doctor,Fabric Shop,Fast Food Restaurant,Field,Flower Shop,Food,Fried Chicken Joint,Gas Station,Gay Bar,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hot Dog Joint,Hotel,Ice Cream Shop,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Jewelry Store,Lake,Lawyer,Library,Light Rail Station,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Music Venue,New American Restaurant,Nightclub,Nightlife Spot,Optical Shop,Other Repair Shop,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Post Office,Print Shop,Pub,Rental Car Location,Resort,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Supermarket,Sushi Restaurant,Taco Place,Thai Restaurant,Theater,Thrift / Vintage Store,Toll Plaza,Trail,Train Station,Video Store,Weight Loss Center,Wine Bar,Yoga Studio
0,Argonne,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
1,Bedford Park,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.0000,0.166667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,0.00,0.000000,0.00,0.000,0.000000,0.0,0.166667,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Beecher,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.166667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.166667,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

#### Print each neighbood along with top 5 venues

In [97]:
num_top_venues = 5

for city in chicago_grouped['City']:
    print("----"+city+"----")
    temp = chicago_grouped[chicago_grouped['City'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Argonne----
               venue  freq
0      Train Station   1.0
1                ATM   0.0
2             Lounge   0.0
3               Park   0.0
4  Other Repair Shop   0.0


----Bedford Park----
                     venue  freq
0                     Park  0.17
1             Intersection  0.17
2  Health & Beauty Service  0.17
3               Print Shop  0.17
4              Beer Garden  0.17


----Beecher----
                  venue  freq
0  Fast Food Restaurant  0.33
1        Discount Store  0.17
2         Bowling Alley  0.17
3           Golf Course  0.17
4                Lawyer  0.17


----Blue Island----
                  venue  freq
0                   Bar  0.31
1        Discount Store  0.08
2                 Diner  0.08
3  Fast Food Restaurant  0.08
4           Flower Shop  0.08


----Bolingbrook----
                     venue  freq
0              Golf Course   0.4
1      American Restaurant   0.2
2               Playground   0.2
3  New American Restaurant   0.2
4             

#### Function to sort venues in descending order

In [98]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### create new datarframe with top 10 venues for each suburb

In [99]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = chicago_grouped['City']

for ind in np.arange(chicago_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(chicago_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head(40)

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Argonne,Train Station,Yoga Studio,Food,Diner,Discount Store,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Electronics Store
1,Bedford Park,Health & Beauty Service,Intersection,Print Shop,Beer Garden,Baseball Field,Park,Grocery Store,Gun Shop,Diner,Discount Store
2,Beecher,Fast Food Restaurant,Bowling Alley,Discount Store,Golf Course,Lawyer,Fried Chicken Joint,Dive Spot,Dog Run,Donut Shop,Dry Cleaner
3,Blue Island,Bar,Diner,Flower Shop,Fast Food Restaurant,Gay Bar,Mexican Restaurant,Bowling Alley,Donut Shop,Train Station,Discount Store
4,Bolingbrook,Golf Course,American Restaurant,Playground,New American Restaurant,Yoga Studio,Flower Shop,Discount Store,Dive Spot,Dog Run,Donut Shop
5,Braidwood,Historic Site,Discount Store,Convenience Store,Sandwich Place,Gas Station,Flower Shop,Dive Spot,Dog Run,Donut Shop,Dry Cleaner
6,Channahon,Gym,Yoga Studio,Food,Diner,Discount Store,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Electronics Store
7,Chicago Heights,Home Service,Wine Bar,Nightlife Spot,Convenience Store,Seafood Restaurant,Flower Shop,Discount Store,Dive Spot,Dog Run,Donut Shop
8,Chicago Ridge,Bar,Liquor Store,Taco Place,Park,Business Service,Chinese Restaurant,Post Office,Fast Food Restaurant,Fabric Shop,Eye Doctor
9,Crest Hill,Home Service,Supermarket,Grocery Store,Fabric Shop,Department Store,Optical Shop,Pharmacy,Clothing Store,Business Service,Breakfast Spot


### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [100]:
# set number of clusters
kclusters = 5

chicago_grouped_clustering = chicago_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chicago_grouped_clustering)

# check cluster labels generated for each 
#kmeans.labels_[0:10] 
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 4, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 0], dtype=int32)

#### Create a new dataframe with cities and cluster info

In [101]:
# add clustering labels
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

chicago_merged = chicagosuburbs

# merge chicago_grouped with chicag_venues to add latitude/longitude for each neighborhood
chicago_merged = chicago_merged.join(city_venues_sorted.set_index('City'), on='City')
chicago_merged.dropna(axis=0, inplace=True)
chicago_merged['Cluster Labels']=chicago_merged['Cluster Labels'].astype(np.int32)
chicago_merged.head() # check the last columns
#chicago_merged.dtypes

,Zip Code,City,County,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,60401,Beecher,Will County,41.343966,-87.616389,0,Fast Food Restaurant,Bowling Alley,Discount Store,Golf Course,Lawyer,Fried Chicken Joint,Dive Spot,Dog Run,Donut Shop,Dry Cleaner
1,60402,Forestview,Cook County,41.835216,-87.791373,0,Bar,Park,Brewery,Nightclub,Cocktail Bar,Italian Restaurant,Market,Pool,Pharmacy,Train Station
2,60406,Blue Island,Cook County,41.654945,-87.681952,0,Bar,Diner,Flower Shop,Fast Food Restaurant,Gay Bar,Mexican Restaurant,Bowling Alley,Donut Shop,Train Station,Discount Store
4,60408,Braidwood,Will County,41.263739,-88.215536,0,Historic Site,Discount Store,Convenience Store,Sandwich Place,Gas Station,Flower Shop,Dive Spot,Dog Run,Donut Shop,Dry Cleaner
6,60410,Channahon,Will County,41.423643,-88.201754,0,Gym,Yoga Studio,Food,Diner,Discount Store,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Electronics Store


#### Visualize the map with clusters and current gyms

Larger labels are the suburbs with color indicicating corresponding cluster, smaller green labels are the gyms

In [102]:
# create map
map_clusters_gyms = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map for the clusters
markers_colors = []
for lat, lon, poi, cluster in zip(chicago_merged['Latitude'], chicago_merged['Longitude'], chicago_merged['City'], chicago_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_gyms)
  
# add markers to the map for the gyms
for lat, lng, venue, city in zip(chicago_gyms['Venue Latitude'], chicago_gyms['Venue Longitude'], chicago_gyms['Venue'], chicago_gyms['City']):
    #print(lat, lng, county, city)
    
    label = '{}, {}'.format(venue, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='darkgreen',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters_gyms) 

map_clusters_gyms

#### Examine Clusters - Cluster 1 is the most common and the one to focus on

#### Cluster 1

In [103]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 0, chicago_merged.columns[[1] + list(range(5,chicago_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beecher,0,Fast Food Restaurant,Bowling Alley,Discount Store,Golf Course,Lawyer,Fried Chicken Joint,Dive Spot,Dog Run,Donut Shop,Dry Cleaner
1,Forestview,0,Bar,Park,Brewery,Nightclub,Cocktail Bar,Italian Restaurant,Market,Pool,Pharmacy,Train Station
2,Blue Island,0,Bar,Diner,Flower Shop,Fast Food Restaurant,Gay Bar,Mexican Restaurant,Bowling Alley,Donut Shop,Train Station,Discount Store
4,Braidwood,0,Historic Site,Discount Store,Convenience Store,Sandwich Place,Gas Station,Flower Shop,Dive Spot,Dog Run,Donut Shop,Dry Cleaner
6,Channahon,0,Gym,Yoga Studio,Food,Diner,Discount Store,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Electronics Store
7,Chicago Heights,0,Home Service,Wine Bar,Nightlife Spot,Convenience Store,Seafood Restaurant,Flower Shop,Discount Store,Dive Spot,Dog Run,Donut Shop
8,Chicago Ridge,0,Bar,Liquor Store,Taco Place,Park,Business Service,Chinese Restaurant,Post Office,Fast Food Restaurant,Fabric Shop,Eye Doctor
12,Dwight,0,Gas Station,Insurance Office,Fried Chicken Joint,Discount Store,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Eye Doctor
13,Elwood,0,Art Gallery,Yoga Studio,Fried Chicken Joint,Discount Store,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Eye Doctor
14,Flossmoor,0,Light Rail Station,Yoga Studio,Fried Chicken Joint,Discount Store,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Eye Doctor


#### Cluster 2

In [104]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 1, chicago_merged.columns[[1] + list(range(5,chicago_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Hazel Crest,1,Asian Restaurant,Yoga Studio,Fried Chicken Joint,Discount Store,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Eye Doctor


#### Cluster 3

In [105]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 2, chicago_merged.columns[[1] + list(range(5,chicago_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Dolton,2,Park,Food,Diner,Discount Store,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Eye Doctor
42,Oak Lawn,2,Park,Jewelry Store,Food,Diner,Discount Store,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Electronics Store
49,Olympia Fields,2,Park,Bakery,Food,Diner,Discount Store,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Electronics Store
63,Steger,2,Park,Construction & Landscaping,Music Venue,Department Store,Diner,Discount Store,Dive Spot,Dog Run,Donut Shop,Dry Cleaner


#### Cluster 4

In [106]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 3, chicago_merged.columns[[1] + list(range(5,chicago_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,Palos Hills,3,Bar,Yoga Studio,Fried Chicken Joint,Discount Store,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Eye Doctor


#### Cluster 5

In [107]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 4, chicago_merged.columns[[1] + list(range(5,chicago_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Matteson,4,Rental Car Location,Yoga Studio,Deli / Bodega,Diner,Discount Store,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Electronics Store


## Find the top 20 suburbs with the fewest number of gyms

#### First create a new data frame from current chicago suburbs but add gymcount feature and initialize to 0

In [108]:
newchicagosuburbs=chicagosuburbs
newchicagosuburbs['gymcount'] = 0
newchicagosuburbs.head()

,Zip Code,City,County,Latitude,Longitude,gymcount
0,60401,Beecher,Will County,41.343966,-87.616389,0
1,60402,Forestview,Cook County,41.835216,-87.791373,0
2,60406,Blue Island,Cook County,41.654945,-87.681952,0
3,60407,Braceville,Grundy County,41.234907,-88.261302,0
4,60408,Braidwood,Will County,41.263739,-88.215536,0


#### Group by suburb to get total gyms for each one.

In [109]:
gym_count=chicago_gyms.groupby('City').size().reset_index(name='counts')

In [110]:
gym_count.head()

,City,counts
0,Bedford Park,14
1,Beecher,3
2,Blue Island,2
3,Bolingbrook,12
4,Braidwood,1


#### Add the counts to the full data frame

In [111]:
for city, counts in zip(gym_count['City'], gym_count['counts']):
    index=newchicagosuburbs[newchicagosuburbs['City'] == city].index
    newchicagosuburbs.loc[index, 'gymcount'] = counts

    
newchicagosuburbs.head()

,Zip Code,City,County,Latitude,Longitude,gymcount
0,60401,Beecher,Will County,41.343966,-87.616389,3
1,60402,Forestview,Cook County,41.835216,-87.791373,4
2,60406,Blue Island,Cook County,41.654945,-87.681952,2
3,60407,Braceville,Grundy County,41.234907,-88.261302,0
4,60408,Braidwood,Will County,41.263739,-88.215536,1


#### sort by fewest gyms and print the top 20

In [112]:
newchicagosuburbs.sort_values(by='gymcount', ascending=True, axis=0, inplace=True)
newchicagosuburbs.head(20)

,Zip Code,City,County,Latitude,Longitude,gymcount
31,60442,Wilton Center,Will County,41.392698,-87.963468,0
33,60444,Mazon,Grundy County,41.243115,-88.402732,0
3,60407,Braceville,Grundy County,41.234907,-88.261302,0
62,60474,South Wilmington,Grundy County,41.179801,-88.266441,0
48,60460,Odell,Livingston County,40.998479,-88.536714,0
26,60437,Kinsman,Grundy County,41.162929,-88.560435,0
69,60481,Custer Park,Will County,41.285524,-88.110548,0
16,60424,Gardner,Grundy County,41.164148,-88.327716,0
10,60417,Crete,Will County,41.427585,-87.586149,0
36,60447,Minooka,Grundy County,41.487239,-88.322166,0
